In [51]:
from datetime import date

param_startDate = date.today()

In [52]:
import pandas as pd

data = {
    "Task Name": ["Task 1", "Task 2", "Task 3"],
    "Maximum Resources": [10, 5, 8],
    "Effort": [14, 6, 3],
    "Execution Order": [1, 2, 2]
}

df = pd.DataFrame(data)
df


,Task Name,Maximum Resources,Effort,Execution Order
0,Task 1,10,14,1
1,Task 2,5,6,2
2,Task 3,8,3,2


### Github copilot algo

In [53]:
import pandas as pd
from datetime import timedelta, date

# Create a copy of the original dataframe
schedule = df.copy()

# Initialize the "Start Date" column
schedule['Start Date'] = schedule.apply(lambda row: param_startDate if row['Execution Order'] == 1 else None, axis=1)

schedule = schedule.sort_values('Execution Order')
schedule


# Calculate the "Finish Date" for each task
for index, row in schedule.iterrows():
    # Check if the task has dependencies
    if row['Execution Order'] > 1:
        # Find the maximum "Finish Date" of the dependent tasks
        dependencies = schedule.loc[schedule['Execution Order'] < row['Execution Order']]
        max_finish_date = dependencies['Finish Date'].max()
        
        # Calculate the "Start Date" based on the maximum "Finish Date" of dependencies
        if pd.notnull(max_finish_date):
            start_date = max_finish_date + timedelta(days=1)
            
            # Skip weekends (Saturday and Sunday)
            while start_date.weekday() >= 5:
                start_date += timedelta(days=1)
            
            schedule.at[index, 'Start Date'] = start_date
            ##print(schedule)
    
    # Calculate the "Finish Date" based on the "Start Date" and "Effort"
    start_date = schedule.at[index, 'Start Date']
    effort = row['Effort']
    finish_date = start_date + timedelta(days=effort)
    
    # Skip weekends (Saturday and Sunday)
    while finish_date.weekday() >= 5:
        finish_date += timedelta(days=1)
    
    schedule.at[index, 'Finish Date'] = finish_date

schedule


,Task Name,Maximum Resources,Effort,Execution Order,Start Date,Finish Date
0,Task 1,10,14,1,2024-03-09,2024-03-25
1,Task 2,5,6,2,2024-03-26,2024-04-01
2,Task 3,8,3,2,2024-03-26,2024-03-29


### TODO: implement Prioirity based task scheduling
The goal is to:
 - Execute the tasks in execution order
 - Minimize Finish time
 - Ignore Max resources for now.